In [16]:
import dataBuilder as db
import numpy as np
import os
import time
import shakespearianModelOrion as smo

import openai



In [17]:
datasetBuilder = db.dataTokenizerBuilder(filename='shakespear.txt', path_to_file='https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt', seq_length=100)


In [18]:
BATCHSIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 1000


dataset = datasetBuilder.build_dataset(batch_size=BATCHSIZE, buffer_size=BUFFER_SIZE)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [19]:
ids_from_chars = datasetBuilder.get_char2idx()

vocab_size = len(ids_from_chars.get_vocabulary())



embedding_dim = 256

rnn_units = 1024



In [20]:
model = smo.Orion(vocab_size, embedding_dim, rnn_units)


In [21]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [22]:
model.summary()

Model: "orion_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     multiple                  16896     
                                                                 
 gru_1 (GRU)                 multiple                  3938304   
                                                                 
 dense_1 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
sample_indices = smo.tf.random.categorical(example_batch_predictions[0], num_samples=1)
sample_indices = smo.tf.squeeze(sample_indices, axis=-1).numpy()


In [24]:
sample_indices

array([32,  1, 25, 38, 56, 34, 45, 62, 27, 60, 45, 57, 48, 12, 20, 25, 23,
       58,  7, 19, 22, 34, 44,  7,  6,  7, 46, 25, 55,  6, 58, 42, 48,  3,
       55, 26, 26, 29,  0, 26, 47,  3, 50, 18, 17, 56, 35, 47,  8, 15, 40,
       40, 52, 29, 55, 33, 54, 23, 37, 64, 23, 30, 36, 51, 62, 54, 46, 56,
       37, 19, 37, 42, 10, 62, 47, 30,  2,  4, 37, 50, 36, 19, 39, 22, 35,
        9, 34, 13, 49, 32, 42, 53, 43,  0, 61, 44, 26, 44, 48, 40],
      dtype=int64)

In [25]:
loss = smo.tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)


Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.1913843


In [26]:
smo.tf.exp(example_batch_loss).numpy()


66.11425

In [27]:
model.compile(optimizer='adam', loss=loss)


In [28]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = smo.os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = smo.tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)


In [29]:
EPOCHS = 20

In [30]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20


172/172 [==============================] - 162s 933ms/step - loss: 2.7046
Epoch 2/20
172/172 [==============================] - 154s 897ms/step - loss: 1.9955
Epoch 3/20
172/172 [==============================] - 159s 924ms/step - loss: 1.7397
Epoch 4/20
172/172 [==============================] - 157s 910ms/step - loss: 1.5774
Epoch 5/20
172/172 [==============================] - 155s 899ms/step - loss: 1.4713
Epoch 6/20
172/172 [==============================] - 154s 896ms/step - loss: 1.4004
Epoch 7/20
172/172 [==============================] - 151s 878ms/step - loss: 1.3449
Epoch 8/20
172/172 [==============================] - 154s 894ms/step - loss: 1.2989
Epoch 9/20
172/172 [==============================] - 158s 920ms/step - loss: 1.2567
Epoch 10/20
172/172 [==============================] - 151s 874ms/step - loss: 1.2161
Epoch 11/20
172/172 [==============================] - 151s 878ms/step - loss: 1.1767
Epoch 12/20
172/172 [==============================] - 150s 871ms/step - l

In [31]:
model.save('OrionModel.keras')

In [32]:

one_step_model = smo.singleStep(model, datasetBuilder.get_idx2char(), datasetBuilder.get_char2idx())



def mainLoop(LEN=800):
    start = time.time()
    states = None
    next_char = smo.tf.constant(['ROMEO:'])
    result = [next_char]

    for i in range(LEN):
        next_char, states = one_step_model.gen_step(next_char, states=states)
        result.append(next_char)
    result = smo.tf.strings.join(result)
    end = time.time()
    text = result[0].numpy().decode('utf-8')

    with open("shakespeare.txt", "w") as f:
        f.write(text)
    return text


In [33]:
mainLoop(LEN=1000)

"ROMEO:\nSo say I read to her:\nWidow, I spy abuse: more present you will lack\nAnd make the learn and ox our lands and\nenvyorn counsel! to bring him not\nUpon my garliam-like a footman by:\nby me trow, this is a Jaunt of wooing,\nOn seduced with the law hath forsor ever\nI was agains with invises; once\nand get him hence in one of Paulina,\nFor such a life, above the minsmen, clay\nand come against your cousin?\n\nISABELLA:\nI am fair a head so sir;\nNay, therefore follower more villain!\n\nDUKE VINCENTIO:\n\nHORTENSIO:\nI do not: mose keen in a made,\nAnd then I'll have them must die time of kings\nand treason born: so strive me one.\n\nLUCENTIO:\nAway! so.\nWho was A noble gentleman and a\nhorse, misinks, for a fellor more, name\nI come abatement where stands you are.\n\nISABELLA:\nI am sweet you, sir; and, none, but I bid thee, sir:\nHear me, you mean now to command me will\nI shall procease with cloids in Such and mendon.\n\nDUKE VINCENTIO:\nMore than that officer, and they will 

In [39]:
import openai

API_KEY = open("APIKEY.txt", "r").read()

client = openai.OpenAI(
    api_key=API_KEY
)


In [40]:
text_to_send = open("shakespeare.txt", "r").read()


content = mainLoop(LEN=1000)



response = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "You are a theater narrator that will be given a text, and have to create a setting, a scene and a context for the dialogues given pretending it's shakespear text, do not include the dialogues in the answer"},
        {"role": "user", "content": text_to_send},
    ],
    model="gpt-3.5-turbo",
    )

print(response.choices[0].message.content)
print("\n\nDialogue:\n\n", text_to_send)

# Define the directory
directory = "history"

# Check if the directory exists, if not, create it
if not os.path.exists(directory):
    os.makedirs(directory)

# Get the list of files in the directory
files = os.listdir(directory)

# If there are no files in the directory, create a new file named tmp0
if not files:
    new_file_name = "tmp0"
else:
    # If there are files, find the highest numbered file and increment its number for the new file
    highest_num = max(int(file[3:]) for file in files if file.startswith("tmp"))
    new_file_name = f"tmp{highest_num + 1}"

# Create the new file in the directory
new_file_path = os.path.join(directory, new_file_name)
with open(new_file_path, 'w') as new_file:
    pass

# Move the content from shakespear.txt to the new file
with open("shakespeare.txt", 'r') as old_file:
    content = old_file.read()
with open(new_file_path, 'w') as new_file:
    new_file.write(content)


**Setting:**  
The curtains rise to reveal a grand theater stage designed to resemble a medieval European town square. Stone buildings with flickering lanterns frame the backdrop, creating a cozy and mysterious atmosphere. A soft glow from the moonlight filters through the windows, casting shadows on the cobbled floor below. The sound of distant laughter and clinking of glasses can be heard from a nearby tavern, adding to the ambiance of the scene.

**Scene:**  
In the center of the stage, Romeo and Bianca stand facing each other, their expressions filled with passion and a hint of conflict. Bianca's handmaid, Kate, lingers close by, her eyes shifting between the two lovers. Leontes, a nobleman with a stern demeanor, approaches them, a look of curiosity on his face. Meanwhile, a page scurries across the stage, carrying a bundle of letters in hand, eager to deliver them to the rightful recipients. Petruchio and Hortensio engage in a lively discussion, gesturing animatedly as they debate